In [1]:
# Install Kaggle API
!pip install kaggle

# Create directory
!mkdir -p ~/.kaggle

# Copy kaggle.json (must be uploaded first)
!cp /content/kaggle.json ~/.kaggle/

# Change permissions
!chmod 600 ~/.kaggle/kaggle.json

print("Kaggle API Successfully Configured!")

Kaggle API Successfully Configured!


In [2]:
!git clone https://github.com/ultralytics/ultralytics
%cd ultralytics

Cloning into 'ultralytics'...
remote: Enumerating objects: 76162, done.
remote: Counting objects: 100% (160/160), done.
remote: Compressing objects: 100% (113/113), done.
remote: Total 76162 (delta 73), reused 87 (delta 47), pack-reused 76002 (from 2)
Receiving objects: 100% (76162/76162), 40.50 MiB | 14.17 MiB/s, done.
Resolving deltas: 100% (57255/57255), done.
/content/ultralytics


In [3]:
!pip install -e .
import torch
print("GPU:", torch.cuda.get_device_name(0))

Obtaining file:///content/ultralytics
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for ultralytics (pyproject.toml) ... done
  Created wheel for ultralytics: filename=ultralytics-8.3.233-0.editable-py3-none-any.whl size=23167 sha256=07c6f44550a02713ca08b9c450dc7180304f81e2e30d9e085b7bb7c091028f56
  Stored in directory: /tmp/pip-ephem-wheel-cache-u6cxjea9/wheels/60/e0/59/e2f034f296abbdca5c21e3f5be76b9ca685f13c7bd17f8b58c
Successfully built ultralytics
GPU: Tesla T4


In [4]:
import os, shutil

print("🧹 Cleaning datasets directory...")
if os.path.exists('/content/datasets'):
    shutil.rmtree('/content/datasets')

os.makedirs('/content/datasets')

%cd /content/datasets

🧹 Cleaning datasets directory...
/content/datasets


In [5]:
print("⬇️ Downloading Brain Tumor Dataset...")
!kaggle datasets download -d pkdarabi/medical-image-dataset-brain-tumor-detection

⬇️ Downloading Brain Tumor Dataset...
Dataset URL: https://www.kaggle.com/datasets/pkdarabi/medical-image-dataset-brain-tumor-detection
License(s): Attribution 4.0 International (CC BY 4.0)
 52% 153M/297M [00:00<00:00, 1.59GB/s]
100% 297M/297M [00:00<00:00, 963MB/s] 


In [6]:
print("📦 Unzipping...")
!unzip -q medical-image-dataset-brain-tumor-detection.zip
!rm medical-image-dataset-brain-tumor-detection.zip

📦 Unzipping...


In [7]:
import os

print("🔍 Locating data.yaml...")

yaml_path = None

for root, dirs, files in os.walk('/content/datasets'):
    if 'data.yaml' in files:
        yaml_path = os.path.join(root, 'data.yaml')
        break

if yaml_path is None:
    raise FileNotFoundError("❌ data.yaml not found!")

dataset_root = os.path.abspath(os.path.dirname(yaml_path))

yaml_content = f"""
path: {dataset_root}
train: train/images
val: valid/images
test: test/images

nc: 3
names: ['glioma', 'meningioma', 'pituitary']
"""

with open(yaml_path, 'w') as f:
    f.write(yaml_content)

print("✅ FIXED data.yaml at:", yaml_path)
print("📍 Dataset root:", dataset_root)

🔍 Locating data.yaml...
✅ FIXED data.yaml at: /content/datasets/BrainTumor/BrainTumorYolov11/data.yaml
📍 Dataset root: /content/datasets/BrainTumor/BrainTumorYolov11


In [8]:
import re

tasks_path = "/content/ultralytics/ultralytics/nn/tasks.py"

simam_code = """
# --- SimAM Attention START ---
# SimAM: A Simple, Parameter-Free Attention Module for CNNs
# Paper: "SimAM: A Simple, Parameter-Free Attention Module for Convolutional Neural Networks"
# This version is parameter-free, uses an energy function per neuron.

class SimAM(nn.Module):
    def __init__(self, c1=None, c2=None, e_lambda=1e-4):
        super().__init__()
        # c1, c2 kept just for YOLO parser compatibility, not used.
        self.e_lambda = e_lambda

    def forward(self, x):
        # x: (B, C, H, W)
        b, c, h, w = x.size()
        n = h * w - 1

        # deviation from mean
        x_minus_mu2 = (x - x.mean(dim=[2, 3], keepdim=True)).pow(2)

        # denominator term: 4 * (var + e_lambda)
        denom = 4 * (x_minus_mu2.sum(dim=[2, 3], keepdim=True) / n + self.e_lambda)

        # energy-based score
        y = x_minus_mu2 / denom + 0.5

        # attention weights (no parameters)
        attn = torch.sigmoid(y)

        return x * attn
# --- SimAM Attention END ---
"""

print("🔧 Injecting SimAM into tasks.py...")

# Read file
with open(tasks_path, "r") as f:
    lines = f.readlines()

# If already present, skip
if any("class SimAM" in line for line in lines):
    print("ℹ️ SimAM already exists in tasks.py, skipping insertion.")
else:
    # Find BaseModel start line
    insert_index = None
    for i, line in enumerate(lines):
        if line.strip().startswith("class BaseModel"):
            insert_index = i
            break

    if insert_index is None:
        print("❌ ERROR: Could not find class BaseModel.")
    else:
        # Insert SimAM block above BaseModel
        new_lines = lines[:insert_index] + [simam_code + "\n"] + lines[insert_index:]
        with open(tasks_path, "w") as f:
            f.writelines(new_lines)
        print("✅ SimAM class inserted above BaseModel.")

        # Register SimAM in YOLO's allowed module list (if not already)
        with open(tasks_path, "r") as f:
            content = f.read()

        if "SimAM," not in content:
            content = content.replace("if m in (", "if m in (SimAM, ", 1)
            with open(tasks_path, "w") as f:
                f.write(content)
            print("✅ SimAM registered in module list.")
        else:
            print("ℹ️ SimAM already registered in module list.")

🔧 Injecting SimAM into tasks.py...
✅ SimAM class inserted above BaseModel.
✅ SimAM registered in module list.


In [9]:
import re

tasks_path = "/content/ultralytics/ultralytics/nn/tasks.py"

simam_code = """
# --- SimAM Attention START ---
# SimAM: A Simple, Parameter-Free Attention Module for CNNs
# Paper: "SimAM: A Simple, Parameter-Free Attention Module for Convolutional Neural Networks"
# This version is parameter-free, uses an energy function per neuron.

class SimAM(nn.Module):
    def __init__(self, c1=None, c2=None, e_lambda=1e-4):
        super().__init__()
        # c1, c2 kept just for YOLO parser compatibility, not used.
        self.e_lambda = e_lambda

    def forward(self, x):
        # x: (B, C, H, W)
        b, c, h, w = x.size()
        n = h * w - 1

        # deviation from mean
        x_minus_mu2 = (x - x.mean(dim=[2, 3], keepdim=True)).pow(2)

        # denominator term: 4 * (var + e_lambda)
        denom = 4 * (x_minus_mu2.sum(dim=[2, 3], keepdim=True) / n + self.e_lambda)

        # energy-based score
        y = x_minus_mu2 / denom + 0.5

        # attention weights (no parameters)
        attn = torch.sigmoid(y)

        return x * attn
# --- SimAM Attention END ---
"""

print("🔧 Injecting SimAM into tasks.py...")

# Read file
with open(tasks_path, "r") as f:
    lines = f.readlines()

# If already present, skip
if any("class SimAM" in line for line in lines):
    print("ℹ️ SimAM already exists in tasks.py, skipping insertion.")
else:
    # Find BaseModel start line
    insert_index = None
    for i, line in enumerate(lines):
        if line.strip().startswith("class BaseModel"):
            insert_index = i
            break

    if insert_index is None:
        print("❌ ERROR: Could not find class BaseModel.")
    else:
        # Insert SimAM block above BaseModel
        new_lines = lines[:insert_index] + [simam_code + "\n"] + lines[insert_index:]
        with open(tasks_path, "w") as f:
            f.writelines(new_lines)
        print("✅ SimAM class inserted above BaseModel.")

        # Register SimAM in YOLO's allowed module list (if not already)
        with open(tasks_path, "r") as f:
            content = f.read()

        if "SimAM," not in content:
            content = content.replace("if m in (", "if m in (SimAM, ", 1)
            with open(tasks_path, "w") as f:
                f.write(content)
            print("✅ SimAM registered in module list.")
        else:
            print("ℹ️ SimAM already registered in module list.")

🔧 Injecting SimAM into tasks.py...
ℹ️ SimAM already exists in tasks.py, skipping insertion.


In [10]:
simam_yaml = """
# YOLOv8n + SimAM (parameter-free attention)

nc: 3  # adjust if friend has different number of classes

backbone:
  - [-1, 1, Conv, [64, 3, 2]]
  - [-1, 1, Conv, [128, 3, 2]]
  - [-1, 3, C2f, [128, True]]
  - [-1, 1, Conv, [256, 3, 2]]
  - [-1, 6, C2f, [256, True]]
  - [-1, 1, Conv, [512, 3, 2]]
  - [-1, 6, C2f, [512, True]]
  - [-1, 1, Conv, [1024, 3, 2]]
  - [-1, 3, C2f, [1024, True]]
  - [-1, 1, SPPF, [1024, 5]]

  # SimAM at deepest feature map (no learnable params, c1 not really used)
  - [-1, 1, SimAM, [1024]]

head:
  - [-1, 1, nn.Upsample, [None, 2, 'nearest']]
  - [[-1, 6], 1, Concat, [1]]
  - [-1, 3, C2f, [512]]

  - [-1, 1, nn.Upsample, [None, 2, 'nearest']]
  - [[-1, 4], 1, Concat, [1]]
  - [-1, 3, C2f, [256]]

  - [-1, 1, Conv, [256, 3, 2]]
  - [[-1, 13], 1, Concat, [1]]
  - [-1, 3, C2f, [512]]

  - [-1, 1, Conv, [512, 3, 2]]
  - [[-1, 10], 1, Concat, [1]]
  - [-1, 3, C2f, [1024]]

  - [[16, 19, 22], 1, Detect, [nc]]
"""

with open("/content/ultralytics/yolov8n-simam.yaml", "w") as f:
    f.write(simam_yaml)

print("✅ Created yolov8n-simam.yaml")

✅ Created yolov8n-simam.yaml


In [12]:
%cd /content/ultralytics
import os
os.environ["WANDB_DISABLED"] = "true"

!yolo detect train \
  data=/content/datasets/BrainTumor/BrainTumorYolov11/data.yaml \
  model=/content/ultralytics/yolov8n-simam.yaml \
  epochs=50 \
  imgsz=640 \
  batch=16 \
  name=YOLOv8_SimAM

/content/ultralytics
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics 8.3.233 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/datasets/BrainTumor/BrainTumorYolov11/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, h